In [1]:
from predict import PredictWNet
from config import Config
import tifffile as tiff
import numpy as np
import sys
import napari

sys.path.append("../..")
from utils import create_dataset_dict_no_labs

c:\Users\colin\anaconda3\envs\celseg\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = Config()
predictor = PredictWNet(
    trained_model_path=r"WNet_results/test_wnet_1500e.pth",
    config=config,
    crf=False,
)

train_files = create_dataset_dict_no_labs(
    volume_directory=r"data/cropped/val/images"
)
train_files = [d.get("image") for d in train_files]
volumes = tiff.imread(train_files).astype(np.float32)

2022-12-21 15:11:53,291 - Images :
2022-12-21 15:11:53,292 - crop_vol_val
2022-12-21 15:11:53,293 - crop_vol_val2
2022-12-21 15:11:53,294 - **********


In [18]:
segmentations = predictor.predict_batch(volumes)
#segmentations = np.where(segmentations > 0.5, 0, 1)
tiff.imwrite(r"WNet_results\segmentations.tiff", segmentations)

In [20]:
segmentations_max = np.argmax(segmentations, axis=1)
segmentation_0 = np.where(segmentations_max == 0, segmentations_max + 1, 0)
segmentation_1 = np.where(segmentations_max == 1, segmentations_max, 0)
segmentation_2 = np.where(segmentations_max == 2, segmentations_max - 1, 0)
print(segmentation_0.shape)

(2, 64, 64, 64)


In [22]:
viewer = napari.view_image(volumes[1], name="Original")
viewer.add_image(segmentation_0[1], name="Segmentation 0")
viewer.add_image(segmentation_1[1], name="Segmentation 1")
viewer.add_image(segmentation_2[1], name="Segmentation 2")

<Image layer 'Segmentation 2' at 0x28a9884f790>

In [5]:
napari.run()

2022-12-21 15:12:25,891 - No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'
